In [81]:
from pathlib import Path
TSV_PATH = Path("/Users/joregan/Playing/hsi/annotations/word_annotations/main")
JSON_PATH = Path("/Users/joregan/Playing/hsi/annotations/final_resolved")

In [40]:
import json

with open("/Users/joregan/Playing/hsi/annotations/all_meta.json") as inf:
    data = json.load(inf)

In [84]:
data["12"]

{'recording_id': 'hsi_3_0715_209_008',
 'segment_id': '42',
 'phrase': 'the Rug',
 'utterance': 'Did you move the Rug like in front of the...',
 'object_id': 'Rug',
 'object_name': 'Rug',
 'frame_number': 2756,
 'image_paths': {'color': 'images/color/hsi_3_0715_209_008_42_color.png',
  'depth': 'images/depth/hsi_3_0715_209_008_42_depth.png',
  'overlay': 'images/overlay/hsi_3_0715_209_008_42_overlay.png',
  'mask': 'images/masks/hsi_3_0715_209_008_42_mask.png'},
 'topic': 'Rug',
 'timing': {'phrase_start': 183.735,
  'phrase_end': 184.295,
  'utterance_start': 183.5558059638532,
  'utterance_end': 195.32878031395185},
 'meta_id': 12}

In [54]:
allsegments = {}

for item in data:
    if not data[item]["recording_id"] in allsegments:
        allsegments[data[item]["recording_id"]] = []
    allsegments[data[item]["recording_id"]].append(data[item]["segment_id"])

for seg in allsegments:
    allsegments[seg].sort(key =lambda x: int(x))

In [ ]:
def get_snippet(data, segment):
    # with open(JSON_PATH / f"{recording}.json") as inf:
    #     data = json.load(inf)
    seg_id = segment["segment_id"]
    seg_start = segment["timing"]["utterance_start"]
    seg_end = segment["timing"]["utterance_end"]
    simple = data[seg_id]
    if simple["general"]["start"] == seg_start and simple["general"]["end"] == seg_end:
        return simple["snippet"]
    else:
        return None

In [85]:
get_snippet(data["12"]["recording_id"], data["12"])

'Did you move the Rug like in front of the...'

In [78]:
def get_topic_context(segments, segment_id, size=None, keep_topic=True):
    current_topic = segments[segment_id]["topic"]
    rec_segments = allsegments[segments[segment_id]["recording_id"]]
    rec_segment_id = segments[segment_id]["segment_id"]
    print(rec_segments, rec_segment_id, segments[segment_id]["recording_id"], segments[segment_id]["segment_id"])

    index = rec_segments.index(rec_segment_id)
    print(index)
    if size is None:
        start = 0
    else:
        start = index - size
    ctx_range = rec_segments[start:index]

    if len(ctx_range) < size:
        if int(segment_id) <= size:
            pass
        else:
            print(f"Warning: size of {size} cannot be satisfied: {range}")
    
    topics = [segments[x]["topic"] for x in ctx_range]
    if keep_topic:
        tmp = []
        for p in zip(ctx_range, topics):
            print(p, segments[p[0]])
            if p[1] == current_topic:
                tmp.append(segments[p[0]]["utterance"])
            else:
                tmp.append(None)
        print(tmp)

In [79]:
get_topic_context(data, "12", 5)

['6', '8', '9', '11', '13', '14', '15', '16', '17', '19', '23', '24', '31', '33', '42', '43', '45', '46', '48', '49', '52'] 42 hsi_3_0715_209_008 42
14
('19', 'fireplace') {'recording_id': 'hsi_3_0715_209_008', 'segment_id': '6', 'phrase': 'the stove', 'utterance': "I see it's a... it's still a in the stove it's a little bit dirty still.", 'object_id': 'Fireplace', 'object_name': 'Fireplace', 'frame_number': 436, 'image_paths': {'color': 'images/color/hsi_3_0715_209_008_6_color.png', 'depth': 'images/depth/hsi_3_0715_209_008_6_depth.png', 'overlay': 'images/overlay/hsi_3_0715_209_008_6_overlay.png', 'mask': 'images/masks/hsi_3_0715_209_008_6_mask.png'}, 'topic': 'fireplace', 'timing': {'phrase_start': 29.117, 'phrase_end': 29.337, 'utterance_start': 29.05712100147809, 'utterance_end': 33.72502066044103}, 'meta_id': 19}
('23', 'rug') {'recording_id': 'hsi_3_0715_210_010', 'segment_id': '11', 'phrase': 'it', 'utterance': 'Yeah, it was a bit expensive, but, yeah, my mother helped me.', 'o